## Fetch Data form Database

In [7]:
import pandas as pd
import numpy as np
import ta,pymongo,dash,os,sys
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.subplots as sp
import yfinance as yf
# Fetch data from database

# Connect to MongoDB
client = pymongo.MongoClient("localhost", 27017)
# Select the database
db = client["streaming_data"]
# Select the collection
collection = db["5m_stock_datastream"]
# Fetch specific stock
data = collection.find({'symbol': 'SAP'})
# Convert to pandas dataframe
df = pd.DataFrame(list(data))

# Set maximum number of columns to display
pd.set_option('display.max_columns', None)  
# Set maximum width for each column
pd.set_option('display.max_colwidth', None)  
# Set maximum width for the entire DataFrame
pd.set_option('display.width', None)  

In [8]:
df = df.sort_values(by='date', ascending=True)
df

,date,symbol,open,low,close,datetime,high,volume,_id
0,2024-10-03 09:30:00,SAP,221.529999,221.170105,221.419998,2024-10-03 09:30:00,221.839996,57128,66ff17b588ff3142ae74e565
1,2024-10-03 09:35:00,SAP,221.419998,220.875000,220.960007,2024-10-03 09:35:00,221.419998,21249,66ff17b588ff3142ae74e566
2,2024-10-03 09:40:00,SAP,221.059998,220.979996,221.367493,2024-10-03 09:40:00,221.380005,16020,66ff17b588ff3142ae74e56a
3,2024-10-03 09:45:00,SAP,221.339996,221.005005,221.250000,2024-10-03 09:45:00,221.410004,14213,66ff17b588ff3142ae74e56b
4,2024-10-03 09:50:00,SAP,221.389999,221.119995,221.160004,2024-10-03 09:50:00,221.389999,5805,66ff17b588ff3142ae74e56c
...,...,...,...,...,...,...,...,...,...
135,2024-10-04 14:15:00,SAP,219.580002,219.475006,219.619995,2024-10-04 14:15:00,219.649994,5903,670031d6169998c9ebb201b5
136,2024-10-04 14:20:00,SAP,219.755005,219.755005,219.949997,2024-10-04 14:20:00,219.970001,2737,67003302169998c9ebb201f2
137,2024-10-04 14:25:00,SAP,219.949997,219.869995,220.009995,2024-10-04 14:25:00,220.009995,2602,6700342f169998c9ebb20236
138,2024-10-04 14:30:00,SAP,220.110001,220.039993,220.039993,2024-10-04 14:30:00,220.110001,1767,6700355a169998c9ebb202ca


In [11]:
# Add project root to sys.path dynamically
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from config.mongdb_config import load_mongo_config

URL = load_mongo_config()['url']
def fetch_data(database, collection, symbol=None,interval=None,datetime=None):
    # Connect to MongoDB
    client = pymongo.MongoClient(URL)
    # Select the database
    db = client[database]
    # Select the collection
    collection = db[collection]
    # Fetch specific stock
    data = collection.find({'symbol': symbol,
                            'interval':interval, 
                            "datetime" : {"$gte":pd.to_datetime(datetime)}}) if symbol else collection.find()
    # Convert to pandas dataframe
    df = pd.DataFrame(list(data))
    return df


In [12]:
fetch_data('data_warehouse', 'live_streaming_data')

,_id,symbol,interval,datetime,alert_type,date
0,6700cef01f3b150689b659ee,QQQ,5m,2024-10-04 10:50:00,bullish_engulfer,2024-10-04 10:50:00
1,6700cef51f3b150689b659f5,QQQ,5m,2024-10-04 11:20:00,bearish_engulfer,2024-10-04 11:20:00
2,6700cef71f3b150689b659f8,QQQ,5m,2024-10-04 11:30:00,bullish_382,2024-10-04 11:30:00
3,6700cf051f3b150689b65a0d,QQQ,5m,2024-10-04 13:00:00,bullish_382,2024-10-04 13:00:00
4,6700cf091f3b150689b65a14,QQQ,5m,2024-10-04 13:20:00,bearish_engulfer,2024-10-04 13:20:00
...,...,...,...,...,...,...
98,6700d18f1f3b150689b65e1b,AAPL,15m,2024-10-04 11:25:00,bullish_382,2024-10-04 11:25:00
99,6700d1991f3b150689b65e2b,TSLA,15m,2024-10-04 10:30:00,bullish_engulfer,2024-10-04 10:30:00
100,6700d1a11f3b150689b65e37,TSLA,15m,2024-10-04 11:20:00,bearish_engulfer,2024-10-04 11:20:00
101,6700d1b51f3b150689b65e55,MSFT,15m,2024-10-04 11:30:00,bullish_382,2024-10-04 11:30:00


## Add Technical Features

In [48]:
# Add bull and bear features in the dataframe
class add_features:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["open"] - self.df["close"])
        self.df["CandleStickType"] = np.where(self.df["open"] < self.df["close"], "green", "red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"close_t-{i}"] = self.df["close"].shift(i)

        self.df['Incremental_High'] = (self.df['close'] > self.df['close_t-1']) \
                                        & (self.df['close_t-1'] > self.df['close_t-2']) \
                                        & (self.df['close_t-2'] > self.df['close_t-3'])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df

    def add_ema_band(self, threshold=0.05):
        self.df['169EMA_Upper'] = self.df['169EMA'] * (1 + threshold)
        self.df['169EMA_Lower'] = self.df['169EMA'] * (1 - threshold)
        return self.df
    
    # Add Technical Indicators
    def add_technical(self):
        
        # Add ema dual channels technical indicators
        self.df['8EMA'] = ta.trend.ema_indicator(self.df['close'], window=8)
        self.df['13EMA'] = ta.trend.ema_indicator(self.df['close'], window=13)
        self.df['144EMA'] = ta.trend.ema_indicator(self.df['close'], window=144)
        self.df['169EMA'] = ta.trend.ema_indicator(self.df['close'], window=169)

        # Add MACD technical indicator
        self.df['MACD'] = ta.trend.macd(self.df['close'], 
                                        window_slow=26, 
                                        window_fast=12)
        
        self.df['MACD_SIGNAL'] = ta.trend.macd_signal(self.df['close'], 
                                                    window_slow=26, 
                                                    window_fast=12, 
                                                    window_sign=9)
        
        self.df['MACD_HIST'] = ta.trend.macd_diff(self.df['close'], 
                                                window_slow=26, 
                                                window_fast=12, 
                                                window_sign=9)
        
        # Add ATR technical indicator
        self.df["atr"] = ta.volatility.AverageTrueRange(high=self.df.high, 
                                                        low=self.df.low, 
                                                        close=self.df.close).average_true_range()
        
        self.df["atr"] = self.df.atr.rolling(window=30).mean()
        
        return self.df
    
    def apply(self):
        self.add_technical()
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        
        # Use last 1000 rows of the dataframe
        self.df = self.df.tail(3000) if len(self.df) > 3000 else self.df
        return self.df
    
testing_df = add_features(df).apply()

## Apply Strategy

In [49]:
class Alert:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['open'].shift(1)
        prev_close = self.df['close'].shift(1)
        
        # Current Candle
        current_open = self.df['open']
        current_close = self.df['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['high'] - self.df['low']
        fib_382_level = self.df['high'] - 0.382 * diff
        open_above_fib_382 = self.df['open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def add_alert(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df
    
# Apply the strategy
testing_df = Alert(testing_df).add_alert()

print(f"Bullish MACD Count: {len(testing_df[testing_df['MACD_Alert'] == 1])} | Bearish Count: {len(testing_df[testing_df['MACD_Alert'] == 0])}")
print(f"Bullish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 1])} | Bearish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 0])}")
print(f"Bullish 382 Count: {len(testing_df[testing_df['382_Alert'] == 1])} | Bearish 382 Count: {len(testing_df[testing_df['382_Alert'] == 0])}")
print(f"Bullish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 1])} | Bearish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 0])}")


Bullish MACD Count: 950 | Bearish Count: 983
Bullish Engulf Count: 90 | Bearish Engulf Count: 125
Bullish 382 Count: 1176 | Bearish 382 Count: 1824
Bullish Dual Channel Count: 804 | Bearish Dual Channel Count: 668


## Simulated trading

In [50]:
import pandas as pd

class TradingStrategy:
    def __init__(self, df, initial_capital=10000):
        self.df = df
        self.initial_capital = initial_capital
        self.trades = []
        self.current_trade = {}
        self.total_values = initial_capital

    def execute_trades(self):
        for date in range(len(self.df) - 1):
            try:
                # Sell Trade
                if len(self.current_trade) != 0:
                    if (self.df['dual_channel_Alert'].iloc[date] == 0) or date == len(self.df) - 1:
                        # Has Trade and Bearish
                        self.trades.append(
                            {
                                "Entry_price": self.current_trade["entry_price"],
                                "Entry_date": self.current_trade["entry_date"],
                                "Exit_price": self.df['open'].iloc[date],
                                "Exit_date": self.df['date'].iloc[date],
                                "profit": (self.df['open'].iloc[date] / self.current_trade['entry_price']) - 1,
                                "total_asset": (self.total_values * (self.df.iloc[date].open / self.current_trade['entry_price']) - 1)
                            }
                        )
                        # Update total asset
                        self.total_values = (self.total_values * (self.df.iloc[date + 1].open / self.current_trade['entry_price']))

                        # Close Trade
                        self.current_trade = {}

                # Buy Trade
                elif (self.df['dual_channel_Alert'].iloc[date] == 1) and (len(self.current_trade) == 0):  # No Trade and Bullish
                    self.current_trade["entry_price"] = self.df['close'].iloc[date]
                    self.current_trade["entry_date"] = self.df['date'].iloc[date]

            except IndexError:
                print("No Trade Data")

    def get_trades(self):
        return pd.DataFrame(self.trades)

    def get_total_return(self):
        if self.trades:
            return self.trades[-1]['total_asset']
        return 0

# Usage

strategy = TradingStrategy(testing_df)
strategy.execute_trades()
trades_df = strategy.get_trades()
total_return = strategy.get_total_return()

print(f"Technical Strategic Total Return: {total_return}")

total_return = 0

Technical Strategic Total Return: 371936.94529403717


## Teesting Static Dashboard

In [51]:
df = collection.find()

# Initialize Dash app
app = dash.Dash(__name__)

# Create figure with subplots
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15, 0.25])

# Define the layout of the Dash app
app.layout = html.Div([
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': symbol, 'value': symbol} for symbol in collection.distinct("symbol")],
        value=collection.distinct("symbol")[0]
    ),
    dcc.Graph(id='sandbox_testing_graph', figure=fig)
])

# Callback to update the graph based on selected stock and selected range
@app.callback(
    Output('sandbox_testing_graph', 'figure'),
    [Input('stock-selector', 'value'), Input('sandbox_testing_graph', 'relayoutData')]
)
def update_chart(selected_stock, relayout_data):
    # Fetch specific stock
    filtered_query = collection.find({"symbol": f"{selected_stock}"})
    # Convert to pandas dataframe
    filtered_df = pd.DataFrame(list(filtered_query))   
    # Add technical features
    filtered_df = add_features(filtered_df).apply()

    # Add Alerts
    filtered_df = Alert(filtered_df).add_alert()
    
    # Sandbox Testing
    trades_history = TradingStrategy(filtered_df)
    trades_history.execute_trades()
    filtered_trades = trades_history.get_trades()
    
    # Create figure with subplots
    fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15, 0.25])

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=filtered_df['date'],
        open=filtered_df['open'],
        high=filtered_df['high'],
        low=filtered_df['low'],
        close=filtered_df['close'],
        name='price'), row=1, col=1)

    # Add EMA traces as lines
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['144EMA'], 
                            mode="lines", name="EMA 144"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['169EMA'],
                            mode="lines", name="EMA 169"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['13EMA'],
                            mode="lines", name="EMA 13"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['8EMA'],
                            mode="lines", name="EMA 8"), row=1, col=1)

    # Add Buy / Sell Annotations
    fig.add_trace(go.Scatter(
                    x=filtered_trades.Entry_date,
                    y=filtered_trades.Entry_price,
                    mode="markers",
                    customdata=filtered_trades,
                    marker_symbol="diamond-dot",
                    marker_size=8,
                    marker_line_width=2,
                    marker_line_color="rgba(0,0,0,0.7)",
                    marker_color="rgba(0,255,0,0.7)",
                    hovertemplate="Entry Time: %{customdata[1]}<br>" +\
                        "Entry Price: %{y:.2f}<br>" +\
                        "Total Asset: %{customdata[5]:.3f}",
                    name="Entries"), row=1, col=1)

    fig.add_trace(go.Scatter(
                    x=filtered_trades.Exit_date,
                    y=filtered_trades.Exit_price,
                    mode="markers",
                    customdata=filtered_trades,
                    marker_symbol="diamond-dot",
                    marker_size=8,
                    marker_line_width=2,
                    marker_line_color="rgba(0,0,0,0.7)",
                    marker_color="rgba(255,0,0,0.7)",
                    hovertemplate="Exit Time: %{customdata[1]}<br>" +\
                        "Exit Price: %{y:.2f}<br>" +\
                        "Total Asset: %{customdata[5]:.3f}",
                    name="Exits"), row=1, col=1)

    # Add MACD subplot
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD'], 
                            mode='lines', name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD_SIGNAL'],
                            mode='lines', name='MACD signal'), row=2, col=1)

    # Add Profit subplot
    fig.add_trace(go.Scatter(x=filtered_trades['Exit_date'], y=filtered_trades['total_asset'], 
                            mode='lines', name='Profit'), row=3, col=1)

    # Layout settings
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        autosize=False,
        width=1000,  # Width of the figure in pixels
        height=800,  # Height of the figure in pixels
    )

    # If a new x-axis range is selected or zoomed
    if relayout_data and 'xaxis.range[0]' in relayout_data and 'xaxis.range[1]' in relayout_data:
        x_start = relayout_data['xaxis.range[0]']
        x_end = relayout_data['xaxis.range[1]']

        # Filter the data based on the selected x-axis range
        filtered_data = filtered_df[(filtered_df['date'] >= x_start) & (filtered_df['date'] <= x_end)]
        trades_filtered = filtered_trades[(filtered_trades['Exit_date'] >= x_start) & (filtered_trades['Exit_date'] <= x_end)]
        
        # Calculate the new y-axis range for the candlestick chart
        y_min = filtered_data['low'].min()
        y_max = filtered_data['high'].max()

        # Calculate the new y-axis range for the MACD chart
        y_min_macd = filtered_data[['MACD', 'MACD_SIGNAL']].min().min()
        y_max_macd = filtered_data[['MACD', 'MACD_SIGNAL']].max().max()

        # Calculate the new y-axis range for the Profit chart
        y_min_profit = trades_filtered['total_asset'].min()
        y_max_profit = trades_filtered['total_asset'].max()

        # Update the figure with new y-axis and x-axis ranges
        fig.update_xaxes(range=[x_start, x_end], row=1, col=1)
        fig.update_xaxes(range=[x_start, x_end], row=2, col=1)  
        fig.update_xaxes(range=[x_start, x_end], row=3, col=1) 
        
        fig.update_yaxes(range=[y_min, y_max], row=1, col=1)
        fig.update_yaxes(range=[y_min_macd, y_max_macd], row=2, col=1)
        fig.update_yaxes(range=[y_min_profit, y_max_profit], row=3, col=1)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[51], line 30, in update_chart(
    selected_stock='GOLD',
    relayout_data=None
)
     28 filtered_df = pd.DataFrame(list(filtered_query))   
     29 # Add technical features
---> 30 filtered_df = add_features(filtered_df).apply()
        filtered_df = Empty DataFrame
Columns: []
Index: []
     32 # Add Alerts
     33 filtered_df = Alert(filtered_df).add_alert()

Cell In[48], line 63, in add_features.apply(self=<__main__.add_features object>)
     62 def apply(self):
---> 63     self.add_technical()
        self = <__main__.add_features object at 0x15b9acca0>
     64     self.add_candlestick()
     65     self.continuous_increase()

Cell In[48], line 33, in add_features.add_technical(self=<__main__.add_features object>)
     30 def add_technical(self):
     31     
     32     # Add ema dual channels technical indicators
---> 3